In [ ]:
import time
import os
import gc
import itertools
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.metrics import mutual_info_score

%matplotlib inline


In [ ]:
data = pd.read_csv('diabetic_data.csv')

In [ ]:
data.shape

In [ ]:
data.drop(['encounter_id', 'patient_nbr'], axis=1 , inplace=True)

In [ ]:
percent_missing = data.isnull().sum() * 100 / len(data)
missing_value_data = pd.DataFrame({'num missing': data.isnull().sum(),
                                 'percent_missing': percent_missing})
missing_value_data.sort_values('percent_missing',ascending=False,inplace=True)
missing_value_data

In [ ]:
(data == 0).astype(int).sum(axis=0)
percent_zero = (data == 0).astype(int).sum(axis=0) * 100 / len(data)
zero_value_data = pd.DataFrame({'num zero': (data == 0).astype(int).sum(axis=0),
                                 'percent_zero': percent_zero})
zero_value_data.sort_values('percent_zero',ascending=False,inplace=True)
zero_value_data

In [ ]:
y = data['diabetesMed']
y.value_counts()

In [ ]:
def preprocess(data):
        numerical = []
        categorical = []
        for f in data.drop(labels=['diabetesMed'], axis=1).columns:
            if data[f].dtype != 'object' and data[f].dtype != 'float64':
                categorical.append(f)
        for f in data.drop(labels=['diabetesMed'], axis=1).columns:
            if data[f].dtype == 'float64':
                numerical.append(f)
        for f in data.drop(labels=['diabetesMed'], axis=1).columns:
            le = preprocessing.LabelEncoder()
            if data[f].dtype == 'object':
                data[f] = le.fit_transform(data[f].apply(str))
                categorical.append(f)
        x = data[numerical + categorical]
        y = data['diabetesMed']
        to_drop = []
        return x, y, numerical, categorical

x, y, numerical, categorical = preprocess(data=data)

In [ ]:
for f in x.columns:
    if mutual_info_score(x[f], y) < 0.0001:
        print(mutual_info_score(x[f], y))
        del x[f]

In [ ]:
train_df, test_df, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=666) # 20 % data is hold out of traning

In [ ]:
train=pd.concat([train_df,y_train], axis=1)
test=pd.concat([test_df,y_test], axis=1)

In [ ]:
train.to_csv('diabete_train.csv')
test.to_csv('diabete_test.csv')